In [11]:
!pip install lambeq
!pip install pytket
!pip install qiskit
!pip install pytket-qiskit
!pip install discopy
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 8.4 MB/s eta 0:00:00


In [155]:
from lambeq import (
    BobcatParser, RemoveCupsRewriter, IQPAnsatz, AtomicType,
    TketModel, QuantumTrainer, SPSAOptimizer,
    Dataset
)
from pytket.extensions.qiskit import AerBackend

import numpy as np
import discopy as dc


In [156]:
import csv

sentences = []
labels = []
with open('sentence_relations_directional.csv', 'r') as file:
      csvreader = csv.reader(file, delimiter=',')
      header = next(csvreader)
      for row in csvreader:
        sentences.append(row[0])
        labels.append(float(row[1]))


In [157]:
parser = BobcatParser()
rewriter = RemoveCupsRewriter()
raw_diagrams = [parser.sentence2diagram(sentence) for sentence in sentences]

rewritten_diagrams = [rewriter(d) for d in raw_diagrams]


In [164]:
ansatz = IQPAnsatz(
    {AtomicType.NOUN: 1, AtomicType.SENTENCE: 1},
    n_layers=2,
    n_single_qubit_params=3
)

input_circuits = [ansatz(d) for d in rewritten_diagrams]
input_labels = []
for label in labels:
    input_labels.append(np.array(label))

train_circuits, train_labels = input_circuits, input_labels

In [165]:
backend = AerBackend()
backend_config = {
    'backend': backend,
    'compilation': backend.default_compilation_pass(2),
    'shots': 8192
}

model = TketModel.from_diagrams(train_circuits, backend_config=backend_config)

In [166]:
BATCH_SIZE = 2
train_dataset = Dataset(train_circuits, train_labels, batch_size=BATCH_SIZE)

In [167]:
bce = lambda y_pred, y_true: -(
    y_true * np.log(y_pred + 1e-10) +
    (1 - y_true) * np.log(1 - y_pred + 1e-10)
).mean()

def accuracy(y_pred, y_true):
    return (np.round(y_pred) == y_true).mean()

eval_metrics = {'accuracy': accuracy}


In [168]:
EPOCHS = 10

trainer = QuantumTrainer(
    model=model,
    loss_function=bce,
    epochs=EPOCHS,
    optimizer=SPSAOptimizer,
    optim_hyperparams={'a': 0.05, 'c': 0.06, 'A': 0.001 * EPOCHS},
    evaluate_functions=eval_metrics,
    evaluate_on_train=True,
    verbose='text',
    log_dir='entailment/logs',
    seed=0
)

trainer.fit(train_dataset)


Epoch 1:   train/loss: 0.8466   valid/loss: -----   train/time: 5.69s   valid/time: -----   train/accuracy: 0.3000   valid/accuracy: -----
Epoch 2:   train/loss: 0.7802   valid/loss: -----   train/time: 5.14s   valid/time: -----   train/accuracy: 0.2500   valid/accuracy: -----
Epoch 3:   train/loss: 1.0577   valid/loss: -----   train/time: 6.99s   valid/time: -----   train/accuracy: 0.3000   valid/accuracy: -----
Epoch 4:   train/loss: 0.7161   valid/loss: -----   train/time: 5.27s   valid/time: -----   train/accuracy: 0.2000   valid/accuracy: -----
Epoch 5:   train/loss: 0.7082   valid/loss: -----   train/time: 6.04s   valid/time: -----   train/accuracy: 0.3500   valid/accuracy: -----
Epoch 6:   train/loss: 0.7066   valid/loss: -----   train/time: 4.98s   valid/time: -----   train/accuracy: 0.2000   valid/accuracy: -----
Epoch 7:   train/loss: 0.6074   valid/loss: -----   train/time: 6.14s   valid/time: -----   train/accuracy: 0.4000   valid/accuracy: -----
Epoch 8:   train/loss: 0.99

In [169]:
#print the outputs on all data points
model.get_diagram_output(train_circuits)

array([[0.24242424, 0.75757576],
       [0.26785714, 0.73214286],
       [1.        , 0.        ],
       [0.17316017, 0.82683983],
       [0.75892857, 0.24107143],
       [0.6744186 , 0.3255814 ],
       [0.86163522, 0.13836478],
       [0.84200743, 0.15799257],
       [0.49553571, 0.50446429],
       [0.73333333, 0.26666667]])